In [ ]:
# if needed, set your working directory to the course example folder
import os
os.chdir(os.path.expanduser('~/scalable-computing-examples'))

In [ ]:
# Set up imports
import pandas as pd
import numpy as np
import xarray as xr

import gcsfs
import fsspec
import zarr

### Create a Zarr dataset

Set a path to serve as the root of the Zarr storage hierarchy, and create a root-level group. Verify that something happened to create `data/example.zarr` on the local file system.

In [ ]:
store = zarr.DirectoryStore('data/example.zarr')
root = zarr.group(store=store, overwrite=True)

Now add two Zarr groups called `temp` and `precip`. We haven't added any data yet! But what changes on the file system?

In [ ]:
root.create_group('temp')
root.create_group('precip')

Next, in the `temp` group, let's define a dataset called `t100`. We'll specify the overall shape of this array, the chunk sizes, the data type, and a fill value for missing data ... but no actual data yet! What does this create under `data/example.zarr` on the file system?


In [ ]:
root.temp.create_dataset('t100',
                         shape=(10000, 10000),
                         chunks=(1000, 1000),
                         dtype='i4',
                         fill_value=99,
                         overwrite=True)

In [ ]:
# Index on the first 5 rows and columns. Does it look like what you expect? Explain!


Within our Python session, now let's create a 10,000 x 10,000 Zarr array of all 1's, specifying a 1K by 1K chunk size. To make it a little more interesting, let's replace the first row to the sequence 0...999. Note that this array only exists in memory for now.

In [ ]:

precip = zarr.ones(shape=(10000, 10000),
                   chunks=(1000, 1000),
                   dtype='i4')
precip[0, :] = np.arange(10000)

In [ ]:
# Index on the first 5 rows and columns. Does it look like what you expect?


Now let's add this precip array to our file-based Zarr store, with the name `p100`. Then look again at the file system. What do you now see under the precip group?

In [ ]:
root.precip['p100'] = precip

Zarr objects in Python have a useful `.info()` method that you can use to retrieve useful operational details about the array (or group). Use this on the stored `t100` and `p100` arrays, as well as your in-memory `precip` array, and consider what it tells you.

In [ ]:
# use the .info() method on Zarr objects

In [ ]:
# Let's go back to our t100 array, and assign 10 across the last
# 3 row by 3 column subarray of the data

# ... then index on the last 5 rows and columns. Does it looks like what you expect?


Now what has changed under example.zarr on the file system? Does it make sense?

For completeness, close the file store connection

In [ ]:
store.close()

Lastly we can read our newly created Zarr store using the `zarr` library, and inspect the data.

In [ ]:
# Use `zarr.open(<path>)` to read in your newly created Zarr store


In [ ]:
# Try using the `tree()` method to visualize its structure


In [ ]:
# Does the precip/p100 array look like what you created?


### Retrieve CMIP6 data from a remote Zarr store

We'll start out by reading a CSV file that catalogs all the available CMIP6 Zarr stores in this Google Cloud Storage account.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

In [ ]:
# Each row identifies a Zarr stores. How many are there in total?


In [ ]:
# What do the first few rows look like?


Next let's use pandas to select CMIP zstore records corresponding to a simulation of the recent past (`historical`) from the ocean daily (`Oday`) table, focusing on the sea surface height (`tos`) variable. We’ll also only select results from the NOAA Geophysical Fluid Dynamics Laboratory (`NOAA-GFDL`) runs.

In [ ]:
df_ta = df.query("""
    activity_id=='CMIP' &
    table_id == 'Oday' &
    variable_id == 'tos' &
    experiment_id == 'historical' &
    institution_id == 'NOAA-GFDL'
    """.replace('\n', ''))
df_ta

The results above are sorted in version order, so we'll just take the final record, and retrieve the Zarr store path.

In [ ]:
zstore = df_ta.zstore.values[-1]
print(f'zstore: {zstore}')

For Zarr, we need to set up a MutableMapping interface to the storage system.

We could use the gcsfs library for this...

In [ ]:
#gcs = gcsfs.GCSFileSystem(token='anon')
#mapper = gcs.get_mapper(zstore)


But instead let's introduce `fsspec`, a useful library that abstracts over many kinds of local and remote connections. Here it detects that we're connecting to GCS, and internally uses the right implementation.


In [ ]:
mapper = fsspec.get_mapper(zstore, token='anon')

Bonus: To get a quick sense of what `fsspec` supports, run these two lines in a code cell:

```python
from fsspec.registry import known_implementations
{k: v.get('class') for k, v in known_implementations.items()}
```

Now let's use xarray to reach into the Zarr store.

Note that this will use Dask by default (if Dask is installed), automagically giving us a lazy, chunked representation of the data See [the docs](https://docs.xarray.dev/en/stable/user-guide/dask.html#reading-and-writing-data) for more information.

In [ ]:
ds = xr.open_zarr(mapper)

In [ ]:
# Have a look at the ds dataset, and specifically the 'tos' data array
# Notice the dimensionality and size.


In [ ]:
# Select a tiny piece of the data, taking a lat slice of 71 to 73,
# lon slice of 203 to 205, and time slice of 2010 to 2012


# How big is this subset of data?

Next let's create a pair of smaller summary datasets, first taking the mean for each day across our returned spatial grid cells, and then taking a 90-day rolling mean over the daily time series.

Importantly, note that because we're working with Dask arrays, these are all lazy computations! We haven't loaded any data yet.

In [ ]:
daily_sst = tos_subset.mean(dim=('lat', 'lon'))
rolling_90d_sst = daily_sst.rolling(time=90, center=True).mean()

Now let's generate a plot, which will force execution and therefore loading of actual data

In [ ]:
daily_sst.plot(label="daily")
rolling_90d_sst.plot(label="rolling annual mean")

As an exercise, you can try adding `.load()` at the end of the assignments above where we first calculated the daily mean and rolling mean, then try plotting again. Notice the speedup in plotting after we've pre-loaded the data in `daily_sst` and `rolling_90d_sst`. In this case, because those summary datasets are so small, this is safe and convenient, especially if we end up re-plotting many times to get the aesthetic details right.

Lastly, let's try writing our small dataset out to a local Zarr store, just to see what it looks like on disk, then read it back in to verify everything is there.

In [ ]:
# Write our original subset out to a local Zarr store
tos_subset.to_zarr('data/cmip_tos_subset.zarr', consolidated=True)

# ... then open it again using xarray
xr.open_dataset('data/cmip_tos_subset.zarr')

## MUR SST

Let's do one more Zarr example, this time pulling SST data from an AWS S3 bucket. We'll use `fsspec` again. Note how similar this is to the code we used to connect to the CMIP6 data in GCS above.

One difference is that here we chose to use the generic `xr.open_dataset()` function rather than `xr.open_zarr()`. Functionally these do the same thing, but the general development trajectory for `xarray` is to favor use of the higher level `open_dataset()` function over the lower-level format-specific functions. When calling this function, we can optionally control Dask chunking using `chunks` argument; here we pass in the value `{}`, which tells `open_dataset()` to choose a chunk layout that preferred by the Zarr engine.

Note: In case your don't have `s3fs` installed in your virtual environment,
you can install it from within a Jupyter cell with the following command:
```python
!pip install s3fs
```

In [ ]:
file_location = 's3://mur-sst/zarr'

mapper = fsspec.get_mapper(file_location, anon=True)
ds_sst = xr.open_dataset(mapper, engine='zarr', consolidated=True, chunks={})

# check out the size of this Zarr array!
ds_sst

In [ ]:
# Select a 1-D time series of the sea_ice_fraction variable array,
# extracting lat 73 and lon -157


# How big is this subset of data?


Let's generate a quick time series plot of the data.

In [ ]:
sea_ice_ts.plot();

In [ ]:
# Now get a chunk of sea_ice_fraction for the year 2015,
# slicing on lat 72.5 to 73, and lon -157.5 to -157

# How big is this subset of data?


Using our subset of data from 2015, calculate monthly mean sea ice fraction for all grid cells in this chunk.

In [ ]:

monthly_mean_ice = sea_ice_chunk.groupby("time.month").mean()
monthly_mean_ice

Finally, let's create a faceted plot visualizing the the ice area fraction by month across our little swath of spatial grid cells

In [ ]:
import matplotlib as mpl
fg = monthly_mean_ice.plot(
    col="month",
    col_wrap=4,
    cmap=mpl.cm.RdYlBu
)